In [1]:
from collections import OrderedDict
import math

from pystencils.session import *
from lbmpy.session import *

from pystencils.boundaries.boundaryhandling import BoundaryHandling

from lbmpy.moments import MOMENT_SYMBOLS

from lbmpy.methods.creationfunctions import create_with_discrete_maxwellian_eq_moments

from lbmpy.phasefield_allen_cahn.parameter_calculation import calculate_parameters_rti
from lbmpy.phasefield_allen_cahn.kernel_equations import *
from lbmpy.phasefield_allen_cahn.force_model import CentralMomentMultiphaseForceModel
from lbmpy.phasefield_allen_cahn.contact_angle import ContactAngle

In [2]:
try:
    import pycuda
except ImportError:
    pycuda = None
    target = ps.Target.CPU
    print('No pycuda installed')

if pycuda:
    target = ps.Target.GPU

No pycuda installed


In [3]:
stencil_phase = LBStencil(Stencil.D2Q9)
stencil_hydro = LBStencil(Stencil.D2Q9)
fd_stencil = LBStencil(Stencil.D2Q9)
assert stencil_phase.D == stencil_hydro.D

dimensions = stencil_phase.D

In [4]:
# domain 
L0 = 256
domain_size = (L0, 4 * L0)
# time step
timesteps = 1000

# reference time
reference_time = 8000
# density of the heavier fluid
rho_H = 1.0

# calculate the parameters for the RTI
parameters = calculate_parameters_rti(reference_length=L0,
                                      reference_time=reference_time,
                                      density_heavy=rho_H,
                                      capillary_number=0.44,
                                      reynolds_number=3000,
                                      atwood_number=0.998,
                                      peclet_number=1000,
                                      density_ratio=1000,
                                      viscosity_ratio=100)
# get the parameters
rho_L = parameters.get("density_light")

mu_H = parameters.get("dynamic_viscosity_heavy")
mu_L = parameters.get("dynamic_viscosity_light")

tau_H = parameters.get("relaxation_time_heavy")
tau_L = parameters.get("relaxation_time_light")

sigma = parameters.get("surface_tension")
M = parameters.get("mobility")
gravitational_acceleration = parameters.get("gravitational_acceleration")


drho3 = (rho_H - rho_L)/3
# interface thickness
W = 5
# coeffcient related to surface tension
beta = 12.0 * (sigma/W)
# coeffcient related to surface tension
kappa = 1.5 * sigma*W
# relaxation rate allen cahn (h)
w_c = 1.0/(0.5 + (3.0 * M))

In [5]:
dh = ps.create_data_handling((domain_size), periodicity=(True, False), parallel=False, default_target=target)

g = dh.add_array("g", values_per_cell=len(stencil_hydro))
dh.fill("g", 0.0, ghost_layers=True)
h = dh.add_array("h",values_per_cell=len(stencil_phase))
dh.fill("h", 0.0, ghost_layers=True)

g_tmp = dh.add_array("g_tmp", values_per_cell=len(stencil_hydro))
dh.fill("g_tmp", 0.0, ghost_layers=True)
h_tmp = dh.add_array("h_tmp",values_per_cell=len(stencil_phase))
dh.fill("h_tmp", 0.0, ghost_layers=True)

u = dh.add_array("u", values_per_cell=dh.dim)
dh.fill("u", 0.0, ghost_layers=True)

C = dh.add_array("C")
dh.fill("C", 0.0, ghost_layers=True)
C_tmp = dh.add_array("C_tmp")
dh.fill("C_tmp", 0.0, ghost_layers=True)

In [6]:
tau = 0.5 + tau_L + (C.center) * (tau_H - tau_L)
s8 = 1/(tau)

rho = rho_L + (C.center) * (rho_H - rho_L)

body_force = [0, 0, 0]
body_force[1] = gravitational_acceleration * rho

In [7]:
config = ps.CreateKernelConfig(target=dh.default_target, cpu_openmp=True)

In [8]:
config_phase = LBMConfig(stencil=stencil_phase, method=Method.CENTRAL_MOMENT, compressible=True,
                         relaxation_rate=1, equilibrium_order=4)

config_hydro = LBMConfig(stencil=stencil_hydro, method=Method.CENTRAL_MOMENT, compressible=False,
                         relaxation_rate=s8, equilibrium_order=4)

In [9]:
method_phase = create_lb_method(lbm_config=config_phase)
method_phase.set_first_moment_relaxation_rate(w_c)

method_hydro = create_lb_method(lbm_config=config_hydro)

In [10]:
# initialize the domain
def Initialize_distributions():
    Nx = domain_size[0]
    Ny = domain_size[1]
    
    for block in dh.iterate(ghost_layers=True, inner_ghost_layers=False):
        x = np.zeros_like(block.midpoint_arrays[0])
        x[:, :] = block.midpoint_arrays[0]
        
        y = np.zeros_like(block.midpoint_arrays[1])
        y[:, :] = block.midpoint_arrays[1]

        y -= 2 * L0
        tmp = 0.1 * Nx * np.cos((2 * math.pi * x) / Nx)
        init_values = 0.5 + 0.5 * np.tanh((y - tmp) / (W / 2))
        block["C"][:, :] = init_values
        
    if target == ps.Target.GPU:
        dh.all_to_gpu()            
    
    dh.run_kernel(h_init)
    dh.run_kernel(g_init)

In [11]:
h_updates = initializer_kernel_phase_field_lb(h, C, u, method_phase, W, fd_stencil=fd_stencil)
g_updates = initializer_kernel_hydro_lb(g, u, method_hydro)

h_init = ps.create_kernel(h_updates, config=config).compile()
g_init = ps.create_kernel(g_updates, config=config).compile()

In [12]:
force_h = [f / 3 for f in interface_tracking_force(C, stencil_phase, W, fd_stencil=stencil_phase)]
force_model_h = CentralMomentMultiphaseForceModel(force=force_h)

force_g = hydrodynamic_force(g, C, method_hydro, tau, rho_H, rho_L, kappa, beta, body_force)
force_model_g = CentralMomentMultiphaseForceModel(force=force_g, rho=rho)

In [13]:
allen_cahn_lb = get_collision_assignments_phase(lb_method=method_phase,
                                                velocity_input=u,
                                                output={'density': C_tmp},
                                                force_model=force_model_h,
                                                symbolic_fields={"symbolic_field": h,
                                                                 "symbolic_temporary_field": h_tmp},
                                                kernel_type='stream_pull_collide')

allen_cahn_lb = sympy_cse(allen_cahn_lb)

ast_allen_cahn_lb = ps.create_kernel(allen_cahn_lb, config=config)
kernel_allen_cahn_lb = ast_allen_cahn_lb.compile()

In [14]:
hydro_lb_update_rule = get_collision_assignments_hydro(lb_method=method_hydro,
                                                       density=rho,
                                                       velocity_input=u,
                                                       force_model=force_model_g,
                                                       sub_iterations=2,
                                                       symbolic_fields={"symbolic_field": g,
                                                                        "symbolic_temporary_field": g_tmp},
                                                       kernel_type='collide_stream_push')

hydro_lb_update_rule = sympy_cse(hydro_lb_update_rule)

ast_hydro_lb = ps.create_kernel(hydro_lb_update_rule, config=config)
kernel_hydro_lb = ast_hydro_lb.compile()

In [15]:
# periodic Boundarys for g, h and C
periodic_BC_C = dh.synchronization_function(C.name, target=dh.default_target, optimization = {"openmp": True})

periodic_BC_g = LBMPeriodicityHandling(stencil=stencil_hydro, data_handling=dh, pdf_field_name=g.name,
                                       streaming_pattern='push')
periodic_BC_h = LBMPeriodicityHandling(stencil=stencil_phase, data_handling=dh, pdf_field_name=h.name,
                                       streaming_pattern='pull')

# No slip boundary for the phasefield lbm
bh_allen_cahn = LatticeBoltzmannBoundaryHandling(method_phase, dh, 'h',
                                                 target=dh.default_target, name='boundary_handling_h',
                                                 streaming_pattern='pull')

# No slip boundary for the velocityfield lbm
bh_hydro = LatticeBoltzmannBoundaryHandling(method_hydro, dh, 'g' ,
                                            target=dh.default_target, name='boundary_handling_g',
                                            streaming_pattern='push')

contact_angle = BoundaryHandling(dh, C.name, fd_stencil, target=dh.default_target)

contact = ContactAngle(45, W)
wall = NoSlip()

if dimensions == 2:
    bh_allen_cahn.set_boundary(wall, make_slice[:, 0])
    bh_allen_cahn.set_boundary(wall, make_slice[:, -1])

    bh_hydro.set_boundary(wall, make_slice[:, 0])
    bh_hydro.set_boundary(wall, make_slice[:, -1])
    
    contact_angle.set_boundary(contact, make_slice[:, 0])
    contact_angle.set_boundary(contact, make_slice[:, -1])
else:
    bh_allen_cahn.set_boundary(wall, make_slice[:, 0, :])
    bh_allen_cahn.set_boundary(wall, make_slice[:, -1, :])

    bh_hydro.set_boundary(wall, make_slice[:, 0, :])
    bh_hydro.set_boundary(wall, make_slice[:, -1, :])
    
    contact_angle.set_boundary(contact, make_slice[:, 0, :])
    contact_angle.set_boundary(contact, make_slice[:, -1, :])


bh_allen_cahn.prepare()
bh_hydro.prepare()

In [16]:
# definition of the timestep for the immiscible fluids model
def Improved_PhaseField_h_g():
    periodic_BC_h()
    bh_allen_cahn()
    
    # run the phase-field LB
    dh.run_kernel(kernel_allen_cahn_lb)
    dh.swap("C", "C_tmp")
    contact_angle()
    # periodic BC of the phase-field
    periodic_BC_C()
    
    dh.run_kernel(kernel_hydro_lb)
    periodic_BC_g()
    bh_hydro()

    # field swaps
    dh.swap("h", "h_tmp")
    dh.swap("g", "g_tmp")

In [17]:
Initialize_distributions()

for i in range(0, timesteps): 
    Improved_PhaseField_h_g()

In [18]:
if target == ps.Target.GPU:
    dh.to_cpu("C")

Ny = domain_size[1]

pos_liquid_front = (np.argmax(dh.cpu_arrays["C"][L0//2, :] > 0.5) - Ny//2)/L0
pos_bubble_front = (np.argmax(dh.cpu_arrays["C"][0, :] > 0.5) - Ny//2)/L0

In [19]:
assert(np.isclose(pos_liquid_front, -1e-1, atol=0.01))
assert(np.isclose(pos_bubble_front, 9e-2, atol=0.01))